<a href="https://colab.research.google.com/github/aparnavinayankozhipuram/RAG-EVALUATION/blob/main/14_April_MIT_RAG_Score_Evaluation_1st_query_3rd_iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nltk rouge-score
!pip install scikit-learn


  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c60400695e3e5fa5127b8665f28001210aaf93f87405f7cffc9a0d3151e90f4d
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [2]:
import nltk
from rouge_score import rouge_scorer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

# Ensure necessary NLTK data is downloaded
# The original line was: nltk.download('punkt')
nltk.download('punkt_tab') # Download punkt_tab instead of just punkt


def evaluate_rag_model(predictions, references):
    """
    Evaluate RAG model using BLEU and ROUGE scores.

    Args:
    predictions (list of str): The generated text from the RAG model.
    references (list of str): The reference ground truth text.

    Returns:
    dict: BLEU and ROUGE scores.
    """
    # BLEU score
    bleu_scores = []
    for prediction, reference in zip(predictions, references):
        reference_tokens = nltk.word_tokenize(reference.lower())
        prediction_tokens = nltk.word_tokenize(prediction.lower())

        # Use smoothing function for BLEU
        smoothie = SmoothingFunction().method1
        bleu_score = sentence_bleu([reference_tokens], prediction_tokens, smoothing_function=smoothie)
        bleu_scores.append(bleu_score)

    avg_bleu_score = sum(bleu_scores) / len(bleu_scores) if bleu_scores else 0

    # ROUGE score
    rouge_scorer_instance = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    rouge_scores = {'rouge1': [], 'rouge2': [], 'rougeL': []}

    for prediction, reference in zip(predictions, references):
        scores = rouge_scorer_instance.score(reference, prediction)
        rouge_scores['rouge1'].append(scores['rouge1'].fmeasure)
        rouge_scores['rouge2'].append(scores['rouge2'].fmeasure)
        rouge_scores['rougeL'].append(scores['rougeL'].fmeasure)

    avg_rouge_scores = {
        'rouge1': sum(rouge_scores['rouge1']) / len(rouge_scores['rouge1']),
        'rouge2': sum(rouge_scores['rouge2']) / len(rouge_scores['rouge2']),
        'rougeL': sum(rouge_scores['rougeL']) / len(rouge_scores['rougeL']),
    }

    # Final Results
    results = {
        'avg_bleu_score': avg_bleu_score,
        'avg_rouge1_score': avg_rouge_scores['rouge1'],
        'avg_rouge2_score': avg_rouge_scores['rouge2'],
        'avg_rougeL_score': avg_rouge_scores['rougeL'],
    }

    return results


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [3]:

# Example usage
predictions = [
    " The three classes of macronutrients are carbohydrates, protein, and fat. Their primary function in the body are as follows:Carbohydrates provide the body with energy by being broken down into glucose, which is then used by the cells for various metabolic processes. Proteins play a crucial role in the growth and repair of tissues, as well as in the production and functioning of enzymes and hormones. Fats are essential for the absorption of fat-soluble vitamins and the formation of cell membranes, as well as for energy storage."


]
references = ["The three classes of macronutrients are carbohydrates, lipids, and proteins. Their primary function is to be metabolically processed into cellular energy, which the body uses to perform work and carry out basic functions."

]

results = evaluate_rag_model(predictions, references)
print("Evaluation Results:")
print(f"Average BLEU Score: {results['avg_bleu_score']:.4f}")
print(f"Average ROUGE-1 Score: {results['avg_rouge1_score']:.4f}")
print(f"Average ROUGE-2 Score: {results['avg_rouge2_score']:.4f}")
print(f"Average ROUGE-L Score: {results['avg_rougeL_score']:.4f}")

Evaluation Results:
Average BLEU Score: 0.1158
Average ROUGE-1 Score: 0.3967
Average ROUGE-2 Score: 0.1681
Average ROUGE-L Score: 0.2810


In [4]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix
import numpy as np

# Function to calculate Precision, Recall, F1 Score for text comparison
def calculate_metrics(y_true, y_pred):
    """
    Calculate Precision, Recall, and F1 Score between ground truth (y_true) and model-generated (y_pred) text.

    Args:
    - y_true: list of reference text (ground truth)
    - y_pred: list of generated text (model output)

    Returns:
    - precision: Precision score
    - recall: Recall score
    - f1: F1 Score
    """
    # Tokenize and convert texts to a binary representation (1 if word exists in the text, else 0)
    y_true_tokens = [set(true.split()) for true in y_true]
    y_pred_tokens = [set(pred.split()) for pred in y_pred]

    # Flatten the sets of tokens into individual word lists for comparison
    all_words = set([word for sublist in y_true_tokens + y_pred_tokens for word in sublist])

    # Create binary vectors for precision, recall, and F1 score
    y_true_binary = [[1 if word in tokens else 0 for word in all_words] for tokens in y_true_tokens]
    y_pred_binary = [[1 if word in tokens else 0 for word in all_words] for tokens in y_pred_tokens]

    # Convert to numpy arrays for use in scikit-learn metrics
    y_true_array = np.array(y_true_binary)
    y_pred_array = np.array(y_pred_binary)

    # Calculate Precision, Recall, and F1 score using sklearn
    precision = precision_score(y_true_array, y_pred_array, average='micro')
    recall = recall_score(y_true_array, y_pred_array, average='micro')
    f1 = f1_score(y_true_array, y_pred_array, average='micro')

    return precision, recall, f1

In [5]:

# Example ground truth (reference) text
y_true = [
    "The three classes of macronutrients are carbohydrates, lipids, and proteins. Their primary function is to be metabolically processed into cellular energy, which the body uses to perform work and carry out basic functions."

]

# Example generated text (model output)
y_pred = [
    "The three classes of macronutrients are carbohydrates, protein, and fat. Their primary function in the body are as follows:Carbohydrates provide the body with energy by being broken down into glucose, which is then used by the cells for various metabolic processes. Proteins play a crucial role in the growth and repair of tissues, as well as in the production and functioning of enzymes and hormones. Fats are essential for the absorption of fat-soluble vitamins and the formation of cell membranes, as well as for energy storage."

]

precision, recall, f1 = calculate_metrics(y_true, y_pred)

# Calculate Precision, Recall, and F1 Score#
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Precision: 0.2759
Recall: 0.5161
F1 Score: 0.3596


In [6]:
!pip install bert-score
import bert_score

# Reference and generated sentences
reference = ["The three classes of macronutrients are carbohydrates, lipids, and proteins. Their primary function is to be metabolically processed into cellular energy, which the body uses to perform work and carry out basic functions."]

generated = ["The three classes of macronutrients are carbohydrates, protein, and fat. Their primary function in the body are as follows:Carbohydrates provide the body with energy by being broken down into glucose, which is then used by the cells for various metabolic processes. Proteins play a crucial role in the growth and repair of tissues, as well as in the production and functioning of enzymes and hormones. Fats are essential for the absorption of fat-soluble vitamins and the formation of cell membranes, as well as for energy storage."]

# Compute BERTScore
P, R, F1 = bert_score.score(generated, reference, lang="en")

# Print Precision, Recall, F1-score
print(f"Precision: {P.mean():.4f}")
print(f"Recall: {R.mean():.4f}")
print(f"F1-score: {F1.mean():.4f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 73.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 86.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Precision: 0.8832
Recall: 0.9297
F1-score: 0.9059
